In [2]:
import os
from glob import glob
import random
import time
import tensorflow as tf
import datetime
from PIL import Image
import PIL
import random
import numpy as np
from matplotlib.pyplot import imshow
import pandas as pd
from IPython.display import FileLink
import  matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
%matplotlib inline
from IPython.display import display, Image
import matplotlib.image as mpimg
import cv2
import time
import glob
import os
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files
from tensorflow.python.keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.metrics import log_loss

from tensorflow import keras
import tensorflow.python.keras.backend as K
from tensorflow.python.keras import callbacks
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.python.keras.applications.vgg16 import VGG16

In [3]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

Using TensorFlow backend.


In [4]:
dataset = pd.read_csv('input_training/driver_imgs_list.csv')

by_drivers=dataset.groupby('subject')
unique_drivers = by_drivers.groups.keys()

NUMBER_CLASSES = 10

In [5]:
def get_cv2_image(path, img_rows, img_cols, color_type=3):
    if color_type == 1:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    elif color_type == 3:
        img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (img_rows, img_cols)) 
    return img

def load_train(img_rows, img_cols, color_type=3):
    start_time = time.time()
    train_images = [] 
    train_labels = []
    for classed in tqdm(range(NUMBER_CLASSES)):
        print('Loading directory c{}'.format(classed))
        files = glob.glob(os.path.join('input_training', 'train', 'c' + str(classed), '*.jpg'))
        for file in files:
            img = get_cv2_image(file, img_rows, img_cols, color_type)
            train_images.append(img)
            train_labels.append(classed)
    print("Data Loaded in {} second".format(time.time() - start_time))
    return train_images, train_labels

def read_and_normalize_train_data(img_rows, img_cols, color_type):
    X, labels = load_train(img_rows, img_cols, color_type)
    y = np_utils.to_categorical(labels, 10)
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    x_train = np.array(x_train, dtype=np.uint8).reshape(-1,img_rows,img_cols,color_type)
    x_test = np.array(x_test, dtype=np.uint8).reshape(-1,img_rows,img_cols,color_type)
    
    return x_train, x_test, y_train, y_test

# Validation
def load_test(size=200000, img_rows=64, img_cols=64, color_type=3):
    path = os.path.join('input_training', 'test', '*.jpg')
    files = sorted(glob.glob(path))
    X_test, X_test_id = [], []
    total = 0
    files_size = len(files)
    for file in tqdm(files):
        if total >= size or total >= files_size:
            break
        file_base = os.path.basename(file)
        img = get_cv2_image(file, img_rows, img_cols, color_type)
        X_test.append(img)
        X_test_id.append(file_base)
        total += 1
    return X_test, X_test_id

def read_and_normalize_sampled_test_data(size, img_rows, img_cols, color_type=3):
    test_data, test_ids = load_test(size, img_rows, img_cols, color_type)
    
    test_data = np.array(test_data, dtype=np.uint8)
    test_data = test_data.reshape(-1,img_rows,img_cols,color_type)
    
    return test_data, test_ids

img_rows = 64
img_cols = 64
color_type = 1

x_train, x_test, y_train, y_test = read_and_normalize_train_data(img_rows, img_cols, color_type)
print('Train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')

nb_test_samples = 200
test_files, test_targets = read_and_normalize_sampled_test_data(nb_test_samples, img_rows, img_cols, color_type)

activity_map = {'c0': 'Safe driving', 
                'c1': 'Right - Texting', 
                'c2': 'Right - Talking on the phone', 
                'c3': 'Left - Texting', 
                'c4': 'Left - Talking on the phone', 
                'c5': 'Operating the radio', 
                'c6': 'Drinking', 
                'c7': 'Reaching behind', 
                'c8': 'Hair and makeup', 
                'c9': 'Talking to passenger'}

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Loading directory c0


 10%|████████▎                                                                          | 1/10 [00:19<02:53, 19.29s/it]

Loading directory c1


 20%|████████████████▌                                                                  | 2/10 [00:36<02:30, 18.79s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [02:39<00:17, 17.15s/it]

Loading directory c9


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:56<00:00, 17.67s/it]


Data Loaded in 176.6916799545288 second
Train shape: (17939, 64, 64, 1)
17939 train samples


  0%|▏                                                                            | 197/79726 [00:01<10:55, 121.39it/s]


# Start Tuning

In [6]:
def data(x_test, x_train, y_test, y_train):
    return x_test, x_train, y_test, y_train

In [17]:
def model(x_test, x_train, y_test, y_train):
    model = Sequential()
    
    model_choice = {{choice(['one', 'two'])}}
    if model_choice == 'one':
        model.add(Conv2D(64, kernel_size=3, activation='relu',padding='same', input_shape=(img_rows, img_cols, color_type)))
        model.add(Conv2D(128, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(256, kernel_size=3, activation='relu'))
        model.add(Conv2D(512, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    elif model_choice == 'two':
        model.add(Conv2D(64, kernel_size=3, activation='relu',padding='same', input_shape=(img_rows, img_cols, color_type)))
        model.add(Conv2D(128, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(256, kernel_size=3, activation='relu'))
        model.add(Conv2D(512, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Flatten())
    model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0, 1)}}))
    choiceval = {{choice(['one', 'two'])}}
    if choiceval == 'two':
        model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(10, activation='softmax'))
    
    adam = keras.optimizers.Adam(lr=0.001)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],
                  optimizer=adam)
    model.fit(x_train, y_train,
              batch_size=256,
              nb_epoch=15,
              verbose=2,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test, verbose=0)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

x_test, x_train, y_test, y_train = data(x_test, x_train, y_test, y_train)
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      trials=Trials(),
                                      notebook_name='tune_notebook')

>>> Imports:
#coding=utf-8

>>> Hyperas search space:

def get_space():
    return {
        'model_choice': hp.choice('model_choice', ['one', 'two']),
        'Dropout': hp.uniform('Dropout', 0, 1),
        'Dropout_1': hp.uniform('Dropout_1', 0, 1),
        'Dropout_2': hp.uniform('Dropout_2', 0, 1),
        'Dropout_3': hp.uniform('Dropout_3', 0, 1),
        'Dense': hp.choice('Dense', [256, 512,1024]),
        'Dropout_4': hp.uniform('Dropout_4', 0, 1),
        'model_choice_1': hp.choice('model_choice_1', ['one', 'two']),
        'Dense_1': hp.choice('Dense_1', [256, 512,1024]),
        'Dropout_5': hp.uniform('Dropout_5', 0, 1),
    }

>>> Data
1: 
2: 
3: 
4: 
>>> Resulting replaced keras model:

   1: def keras_fmin_fnct(space):
   2: 
   3:     model = Sequential()
   4:     
   5:     model_choice = space['model_choice']
   6:     if model_choice == 'one':
   7:         model.add(Conv2D(64, kernel_size=3, activation='relu',padding='same', input_shape=(img_rows, img_cols, color

job exception: name 'Sequential' is not defined



  0%|                                                                           | 0/30 [00:00<?, ?trial/s, best loss=?]


NameError: name 'Sequential' is not defined